In [6]:
import pickle
import string

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds

import numpy as np
import pandas as pd

import fasttext as ft
import fasttext.util

In [7]:
tf.config.list_physical_devices('GPU'), tf.test.is_built_with_cuda()

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], True)

In [8]:
# Our dataset is around ~15mb
df = pd.read_csv('old_general.csv', header=None, usecols=[2], names=['chat'])
df.head(10)

,chat
0,Nigger in the house
1,Nigger
2,"""Yes"""
3,I have a hard on right now
4,same tbh
5,im actually jerking off rn
6,I'm fapping to cosplaying kids
7,same
8,con season is best season
9,true


In [9]:
UNK = 'xxunk'
# OOV included in encoder

def raw_text_preprocess(x):
    return x.lower().translate(str.maketrans('', '', string.punctuation))

df['chat'] = df['chat'].apply(raw_text_preprocess)

df.head(10)

,chat
0,nigger in the house
1,nigger
2,yes
3,i have a hard on right now
4,same tbh
5,im actually jerking off rn
6,im fapping to cosplaying kids
7,same
8,con season is best season
9,true


In [10]:
df['tkn_len'] = df['chat'].apply(lambda x: len(x.split(' ')))

_p = {i: np.percentile([l for l in df['tkn_len'].values], i) for i in [70, 80, 90, 95, 97, 99]}

_p

{70: 7.0, 80: 9.0, 90: 12.0, 95: 16.0, 97: 19.0, 99: 28.0}

In [11]:
MAX_TOKENS = 20

df = df[df['tkn_len'] <= MAX_TOKENS]

BOS = 'xxbos'
EOS = 'xxeos'

def add_utility_tokens(r):
    content = r['chat']

    return f'{BOS} {content} {EOS}'

df['chat'] = df.apply(add_utility_tokens, axis=1)

MAX_TOKENS += 2

df.head(10)

,chat,tkn_len
0,xxbos nigger in the house xxeos,4
1,xxbos nigger xxeos,1
2,xxbos yes xxeos,1
3,xxbos i have a hard on right now xxeos,7
4,xxbos same tbh xxeos,2
5,xxbos im actually jerking off rn xxeos,5
6,xxbos im fapping to cosplaying kids xxeos,5
7,xxbos same xxeos,1
8,xxbos con season is best season xxeos,5
9,xxbos true xxeos,1


In [12]:
# push to tensorflow
df['chat_shift'] = df['chat'].shift(-1)
df.drop(df.tail(1).index, inplace=True) # drop last row for NaN due to shift

dataset = tf.data.Dataset.from_tensor_slices((df['chat'].values, df['chat_shift'].values))

In [13]:
# tokenize and build vocab
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
vocabulary_set.update([BOS, EOS, UNK])

for xb, yb in dataset:
  xb_tokens = tokenizer.tokenize(xb.numpy())
  vocabulary_set.update(xb_tokens)

  yb_tokens = tokenizer.tokenize(yb.numpy())
  vocabulary_set.update(yb_tokens)

vocab_size = len(vocabulary_set)
vocab_size += 1 # for padding introduced dataset.padded_batch (as 0.0f)

vocab_size

43605

In [14]:
next(iter(dataset.shuffle(10)))[0].numpy()

b'xxbos im fapping to cosplaying kids xxeos'

In [15]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

# collate_fn (for the fastai users)
def encode(xb, yb):
  enc_x, enc_y = encoder.encode(xb.numpy()), encoder.encode(yb.numpy())
  return enc_x, enc_y

def encode_map_fn(xb, yb):
  enc_xb, enc_yb = tf.py_function(encode, 
                                       inp=[xb, yb], 
                                       Tout=(tf.int64, tf.int64))
  return enc_xb, enc_yb

dataset = dataset.map(encode_map_fn)
next(iter(dataset))[0]

<tf.Tensor: shape=(6,), dtype=int64, numpy=array([35301,   579, 36036,  9786, 16519, 31726], dtype=int64)>

In [44]:
BUFFER_SIZE = 1024
BATCH_SIZE = 256

# it's a small dataset in memory, we don't need
# prefetch or caching

# https://www.tensorflow.org/guide/data_performance
train_data = dataset.batch(BATCH_SIZE)
                    .map(encode_map_fn,
                        num_parallel_calls=tf.data.experimental.AUTOTUNE
                        )
                    .unbatch()

NameError: name 'encoded_dataset' is not defined

In [121]:
def keras_dataset_generator(ds, epochs=10):
    _ds = ds.unbatch().repeat(epochs)
    for xb, yb in _ds:
        pad_xb = [[0, MAX_TOKENS-len(xb)]]
        xb_padded = tf.pad(xb, pad_xb, mode='CONSTANT')
        # [enc_input, dec_input, ]
        for i in range(1, len(yb)):
            yield [xb, tf.pad(yb[0:i], [[0, MAX_TOKENS-i]], mode='CONSTANT')], yb[i] 

In [162]:
# load fasttext model and build embeddings layer
# ft.util.download_model('en', if_exists='ignore')
en_vecs = ft.load_model('./cc.en.300.bin')
en_vecs.get_dimension() # 300

# Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.
# you can safely ignore this warning, it's for other model types and (possibly) deprecation warning for other versions

300

In [187]:
embedding_matrix = np.zeros((vocab_size, 300))

for word in vocabulary_set:
    i = encoder.encode(word)
    embedding_matrix[i] = en_vecs.get_word_vector(word)

embedding_layer = tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix], name='embedding')

In [196]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# https://keras.io/examples/lstm_seq2seq/

latent_dim = 128

# input is (tokens, vocab size)
encoder_input = Input(shape=(MAX_TOKENS), name='encoder_input')

# embedding dim is 300 from the fasttext model
encoder_embedding = embedding_layer(encoder_input) # None, MAX_TOKENS, 300

encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

encoder_states = [state_h, state_c]

# # same logic as model_input
decoder_input = Input(shape=(MAX_TOKENS), name='decoder_input')

decoder_embedding = embedding_layer(decoder_input)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_embedding,
                                     initial_state=encoder_states)

decoder_dense = Dense(vocab_size, activation='softmax', name='decoder_dense')
decoder_output = decoder_dense(decoder_outputs)

model = Model([encoder_input, decoder_input], decoder_output)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, 22)]         0                                            
__________________________________________________________________________________________________
encoder_input (InputLayer)      [(None, 22)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             13081500    encoder_input[0][0]              
                                                                 decoder_input[0][0]              
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 128), (None, 219648      embedding[9][0]            

In [19]:
model.fit_generator(keras_dataset_generator(train_ds, epochs=5),
          batch_size=BATCH_SIZE,
          epochs=EPOCHS)

NameError: name 'model' is not defined

In [220]:
# define wiring of sampling models to get some faster inference

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

array([[[6., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]]])

In [ ]:
def decode_sequence(input_seq):
    enc_input_seq = encoder.encode(input_seq)
    
    states_value = encoder_model.predict(input_seq)

    # Batch size is 1 this is why there is an extra sequence
    target_seq = np.zeros((1, 1, MAX_TOKENS))
    
    # sampling recurrent loop

    i = 0
    target_seq[0, 0, i] = encoder.encode(BOS)
    decoded_sentence = ''
    while True:
        i += 1
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # argmax the output to get next token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = encoder.decode[sampled_token_index]

        decoded_sentence += sampled_word

        # if max length or EOS, stop
        if (sampled_char == EOS or
           len(decoded_sentence) > MAX_TOKENS):
            break

        # update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, i] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

In [20]:
# visualize some results from test data

test_data = ['Whatsup bot?', 'marios is good', 'can i get admin pretty pls']

for raw_text in test_data:
    input_text = raw_text_preprocess(raw_text)

    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_text)
    print('Decoded sentence:', decoded_sentence)

In [22]:
# deep learning
encoder_model.save('encoder_model', save_format='tf')
decoder_model.save('encoder_model', save_format='tf')

# vocabulary maps
encoder.save_to_file('text.encoder')